In [1]:
import pandas as pd

from utils import config, utils

In [2]:
df = pd.read_excel(
    "./../../upgrade_mandi/raw-sheets-dump/zepto raw.xlsx",
    sheet_name="Sheet1",
    header=1,
)

In [3]:
df.columns = df.columns.str.strip()

In [4]:
locations = [
    location.locationName for location in config.domainConfigClass["Zepto"].locations
]

In [ ]:
productDF = df[
    df.columns[[not column.startswith("NAG-") for column in df.columns]]
].drop(labels=["Product Code", "Grand Total", "Vendor Name"], axis=1)

productDF["UoM"] = productDF["UoM"].apply(
    lambda uom: "#N/A" if str(uom) == "nan" else uom
)

locationDF = df[df.columns[[column.startswith("NAG-") for column in df.columns]]]
locationDF.columns = [
    utils.nameExtracter(locations, column[4:]) for column in locationDF.columns
]

# locationDF.columns = [ for location in locationDF.columns]

In [6]:
productDF

,Product Name,UoM,Rate
0,Tomato Local,500 GRAM,44
1,Papaya,1 PIECE,55
2,Beans French,#N/A,40
3,Sweet Corn,2 PIECE,25
4,Lady Finger,250 GRAM,15
5,Ivy Gourd (Tindli),250 GRAM,21
6,Colocasia Leaves (Arvi Patta),5 leaves,20
7,Lemon,250 GRAM,32
8,Cucumber (Kheera),2 PIECE,17
9,Pointed Gourd (Parwal),250 GRAM,32


In [20]:
locationSepratedDF = {}
for location in locations:
    locationSeries = locationDF[location]
    locationSeries.name = "Invoice Qty."
    locationSepratedDF[location] = pd.concat([productDF, locationSeries], axis=1)

    locationSepratedDF[location]["Amount"] = (
        locationSepratedDF[location]["Rate"]
        * locationSepratedDF[location]["Invoice Qty."]
    )

    locationSepratedDF[location].columns = [
        "Article Name",
        "UoM",
        "Rate",
        "Invoice Qty.",
        "Amount",
    ]

    locationSepratedDF[location]["Recieved Qty."] = ""
    locationSepratedDF[location]["No"] = range(1, len(locationSepratedDF[location]) + 1)

    locationSepratedDF[location] = locationSepratedDF[location][
        [
            "No",
            "Article Name",
            "UoM",
            "Invoice Qty.",
            "Recieved Qty.",
            "Rate",
            "Amount",
        ]
    ]

locationSepratedDF["Khamala"]

,No,Article Name,UoM,Invoice Qty.,Recieved Qty.,Rate,Amount
0,1,Tomato Local,500 GRAM,51,,44,2244
1,2,Papaya,1 PIECE,0,,55,0
2,3,Beans French,NaN,2,,40,80
3,4,Sweet Corn,2 PIECE,10,,25,250
4,5,Lady Finger,250 GRAM,46,,15,690
5,6,Ivy Gourd (Tindli),250 GRAM,2,,21,42
6,7,Colocasia Leaves (Arvi Patta),5 leaves,1,,20,20
7,8,Lemon,250 GRAM,17,,32,544
8,9,Cucumber (Kheera),2 PIECE,18,,17,306
9,10,Pointed Gourd (Parwal),250 GRAM,2,,32,64
